# Maximum Likelihood Density Ratio Estimation for MI

paper: http://proceedings.mlr.press/v4/suzuki08a/suzuki08a.pdf

In [4]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pycuda.driver as cuda

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
use_cuda = torch.cuda.is_available()
print("Setting Info")
print("=========")
print("- use_cuda: ", use_cuda)
print("- Path: ", os.getcwd())
print("- PyTorch", torch.__version__)
print("- Python: ", sys.version)

Setting Info
- use_cuda:  True
- Path:  /home/uchiumi/JNNS2019/mnist_pytorch
- PyTorch 1.0.1.post2
- Python:  3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]


## Buildiing Network

In [6]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        layer_values={}
        layer_values["input_image"] = x
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        layer_values["conv1_output"] = x
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        layer_values["conv2_output"] = x
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        layer_values["fc1_output"] = x
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        layer_values["fc2_output"] = x
        x = F.log_softmax(x)
        layer_values["output_softmax"] = x
        
        return layer_values

## Load Model

In [7]:
# model reload
model = Network()

PRETRAINED_MODEL_PATH = "/home/uchiumi/JNNS2019/mnist_pytorch/train_log/model__2019-0422-1338.pth"

model.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))

## Data Loader

cf.) https://www.aiworkbox.com/lessons/examine-mnist-dataset-from-pytorch-torchvision

In [8]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [9]:
X_train_0 = np.asarray(mnist_trainset[0][0]) # image
y_train_0 = mnist_trainset[0][1] # label

In [10]:
# train
train_loader_for_MINE = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=False)

In [11]:
# test
test_loader_for_MINE = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=False)

## Get layer values (the state of each nodes)

In [12]:
def get_nodes_with_train_data(model):
    model.eval()
    list = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(train_loader_for_MINE):
            result = model(data)
            list.append(result)
    return list

In [13]:
def get_nodes_with_test_data(model):
    model.eval()
    list = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader_for_MINE):            
            result = model(data)
            list.append(result)
    return list

In [14]:
list = get_nodes_with_train_data(model)

/home/uchiumi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:
len(list)

60000

In [16]:
list[0].keys()

dict_keys(['conv2_output', 'fc2_output', 'output_softmax', 'fc1_output', 'input_image', 'conv1_output'])

In [17]:
print("input_image", list[0]["input_image"].shape)
print(list[0]["conv1_output"].shape)
print(list[0]["conv2_output"].shape)
print(list[0]["fc1_output"].shape)
print(list[0]["fc2_output"].shape)
print(list[0]["output_softmax"].shape)

input_image torch.Size([1, 1, 28, 28])
torch.Size([1, 10, 12, 12])
torch.Size([1, 20, 4, 4])
torch.Size([1, 50])
torch.Size([1, 10])
torch.Size([1, 10])


In [18]:
tensordata_input_image = []
tensordata_conv1_output = []
tensordata_conv2_output = []
tensordata_fc1_output = []
tensordata_fc2_output = []
tensordata_output_softmax = []

for i in range(len(train_loader_for_MINE)):
    tensordata_input_image.append(list[i]["input_image"].data.numpy().flatten())
    tensordata_conv1_output.append(list[i]["conv1_output"].data.numpy().flatten())
    tensordata_conv2_output.append(list[i]["conv2_output"].data.numpy().flatten())
    tensordata_fc1_output.append(list[i]["fc1_output"].data.numpy().flatten())
    tensordata_fc1_output.append(list[i]["fc2_output"].data.numpy().flatten())
    tensordata_output_softmax.append(list[i]["output_softmax"].data.numpy().flatten())

In [19]:
tensordata_input_image = np.array(tensordata_input_image)
tensordata_conv1_output = np.array(tensordata_conv1_output)
tensordata_conv2_output = np.array(tensordata_conv2_output)
tensordata_fc1_output = np.array(tensordata_fc1_output)
tensordata_fc2_output = np.array(tensordata_fc2_output)
tensordata_output_softmax = np.array(tensordata_output_softmax)

## Maximum Likelihood Density Ratio Estimation for MI

In [20]:
x = tensordata_conv1_output
y = tensordata_conv2_output
z = np.concatenate([x, y], axis=1)

In [21]:
print(x.shape)
print(y.shape)
print(z.shape)

(60000, 1440)
(60000, 320)
(60000, 1760)


In [22]:
class RBFkernel():
    def __init__(self, sigma=0.5):
        self.sigma = sigma
        
    def __call__(self, x, y):
        if x.shape[0] != y.shape[0]: 
            raise Exception("please arrange the shape of input arrray.")
            
        self.sample_space_dim = x.shape[0]
        return np.exp(-1 * ( np.sum((x - y)**2 ) / 2 * self.sigma**2))

test

In [25]:
kernel = RBFkernel(sigma=0.5)
kernel(np.array([1, 2, 3]), np.array([6, 7, 8]))

8.481823524646916e-05

In [29]:
class DensityRatioEstimation():
    def __init__(self, x, y, kernel):
        if x.shape[0] != y.shape[0]:
            raise Exception("please arrange the shape of input arrray x and y.")
        self.x = x
        self.y = y
        self.kernel = kernel
        self.sample_size = x.shape[0]
    
    def update(self):
        pass
    
    def fit_alpha(self):
        pass
    